In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [2]:
eg_conn = psrcelmerpy.ElmerGeoConn()


In [3]:
# Load extract of TransRefEdges (provided by Nick)
gdf = gpd.read_file(r'X:\DSA\bike_lts\2023_PedBikeInventory_forLTS_withSlope.shp')
gdf = gdf.to_crs('EPSG:2285')

In [4]:
# Load bike stress table, adapted from Lowry
# https://www1.coe.neu.edu/~pfurth/Furth%20papers/2016%20Prioritizing%20to%20improve%20low-stress%20network%20connectivity%20Lowry,%20Furth,%20Hadden-Loh.pdf

df_data = pd.read_csv('bike_stress_table.csv')

In [5]:
# The Lowry table provides values based on bi-directional data of lanes and speed limit
# The transRefEdges network is made up of directional links, so we need to sum lanes to match Lowry

# Assume speed limit will be the same in both directions, use IJ direction
gdf['SpeedLimit_new'] = gdf['IJSpeedLim'].copy()
gdf.loc[gdf['IJSpeedLim'] <= 25, 'SpeedLimit_new'] = 25
gdf.loc[gdf['IJSpeedLim'] >= 35, 'SpeedLimit_new'] = 35

# Lanes will be the sum of both directions
gdf['lanes_new'] = (gdf['IJLanesGPA']+gdf['JILanesGPA']).astype('int')

# Set maximum lanes at 6 and minimum at 2
gdf.loc[gdf['lanes_new'] >= 6, 'lanes_new'] = 6
gdf.loc[gdf['lanes_new'] <2, 'lanes_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityTy'] == 9, 'LocalStreet'] = 1

In [6]:
# Intersect with urban growth area
gdf_growth_area = eg_conn.read_geolayer('urban_growth_area')
gdf_growth_area = gdf_growth_area[['county_name','geometry']].to_crs('EPSG:2285')

gdf = gpd.sjoin(gdf, gdf_growth_area, how="left")

gdf['rural'] = 'not_rural'
gdf.loc[gdf['county_name'].isnull(),'rural'] = 'rural'

In [7]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    11: 'SharedUsePath'    # Shared use path
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Get area type to be able to classify local streets
gdf['AreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['SpeedLimit_new']>30, 'AreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood regardless of street type
gdf.loc[(gdf['lanes_new']>2, 'AreaType')] = 'Other'

gdf_new = gdf.copy()
for dir in ['IJ','JI']:
    # Create new fields that use the bike facility
    gdf_new[dir+'BikeFacility'] = gdf_new[dir+'BikeType'].map(bike_type_map)

    # If in rural area, treat paved shoulder as bike lanes
    gdf_new.loc[((gdf_new['rural']=='rural')&(gdf_new[dir+'BikeType']==3)), dir+'BikeFacility'] = 'BikeLane'

    # Merge to df_data to get stress values
    gdf_new = gdf_new.merge(df_data, left_on=['SpeedLimit_new', 'lanes_new', dir+'BikeFacility','AreaType'], 
            right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left', suffixes=['_IJ','_JI'])

In [8]:
# Add length in feet
gdf_new['Shape_Length'] = gdf_new.length

In [9]:
# Add impact of slope as an additional stress factor on top of the facility type, speed limit, and number of lanes
gdf_new['upslp'] = gdf_new['Max_Slope']/gdf_new['SLength']
for dir in ['IJ','JI']:
    gdf_new.loc[(gdf_new['upslp'] > .02) & (gdf_new['upslp'] <= .04), 'Factor_'+dir] = gdf_new['Factor_'+dir] + 0.37
    gdf_new.loc[(gdf_new['upslp'] > .04) & (gdf_new['upslp'] <= .06), 'Factor_'+dir] = gdf_new['Factor_'+dir] + 1.2
    gdf_new.loc[(gdf_new['upslp'] > .06), 'Factor_'+dir] = gdf_new['Factor_'+dir] + 3.2

# Assign LTS based on stress factor
for dir in ['IJ','JI']:
    gdf_new.loc[gdf_new['Factor_'+dir] < 0.1, 'LTS_'+dir] = 1
    gdf_new.loc[(gdf_new['Factor_'+dir] >= 0.1) & (gdf_new['Factor_'+dir] < 0.3), 'LTS_'+dir] = 2
    gdf_new.loc[(gdf_new['Factor_'+dir] >= 0.3) & (gdf_new['Factor_'+dir] < 0.6), 'LTS_'+dir] = 3
    gdf_new.loc[gdf_new['Factor_'+dir] >= 0.6, 'LTS_'+dir] = 4

In [10]:
# Write to file
gdf_new.to_csv(r'X:\DSA\bike_lts\output\transRefEdges_bike_stress_2023_01_27_25.csv')